# Fractal VR TMT analysis. (in R)

In [ ]:
# libraries

# List of packages to install
packages <- c("ez", "emmeans", "cowplot", "dplyr", "readr", "ggplot2", "lmerTest", 
              "hrbrthemes", "sjPlot", "sjmisc", "sjlabelled", "tidyr", "tidyverse", 
              "ggthemes", "ggpubr", "knitr", "jtools", "apaTables", "psych", "boot")

# Install packages
install.packages(packages)

# load data